# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol ='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data
data['symbol']

'AAPL'

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap =data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker','SharePrice','MarketCap','Number Of Shares']
new_dataFrame = pd.DataFrame(columns = my_columns)

new_dataFrame.append(pd.Series([symbol,price,market_cap,'N/A'],index=my_columns),ignore_index=True)

,Ticker,SharePrice,MarketCap,Number Of Shares
0,AAPL,126.64,2082698142688,N/A


In [7]:
final_dataframe = pd.DataFrame(columns= my_columns)

for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([data['symbol'],data['latestPrice'],
                   data['marketCap'],'N/A'],
                  index=my_columns),
                    ignore_index=True)
    

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [7]:
final_dataframe

NameError: name 'final_dataframe' is not defined

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [8]:
def chunks(lst,n):
    """yields n sized chunks fromt a list"""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        


In [9]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings
final_dataframe2 = pd.DataFrame(columns=my_columns)
# batch api call 100 
for symbol_string in symbol_strings:
    batch_api = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data= requests.get(batch_api).json()
    for symbol in symbol_string.split(','):
        final_dataframe2 = final_dataframe2.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],index=my_columns),
            ignore_index = True
        )
final_dataframe2
    

,Ticker,SharePrice,MarketCap,Number Of Shares
0,A,123.02,37990982273,N/A
1,AAL,17.34,10737796145,N/A
2,AAP,160.22,10878565741,N/A
3,AAPL,127.80,2120186516103,N/A
4,ABBV,107.76,189996347802,N/A
5,ABC,99.97,20500359384,N/A
6,ABMD,265.82,11738671026,N/A
7,ABT,110.40,193783354147,N/A
8,ACN,246.62,163315982952,N/A
9,ADBE,484.06,231583175722,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [13]:
portfolio_size = input("Enter the portfolio size you want to invest: ")

try:
    val = float(portfolio_size)
except ValueError:
    print("this is not a  float or integer")
    portfolio_size = input("Enter the portfolio size you want to invest: ")
    val = float(portfolio_size)


    

Enter the portfolio size you want to invest: ltkghmr
this is not a  float or integer
Enter the portfolio size you want to invest: 1000000


In [21]:
position_size = val/len(final_dataframe2.index)
for i in range(len(final_dataframe2)):
    final_dataframe2.loc[i,'Number Of Shares'] = math.floor(position_size/final_dataframe2.loc[i, 'SharePrice'])
final_dataframe2

,Ticker,SharePrice,MarketCap,Number Of Shares
0,A,123.02,37990982273,16
1,AAL,17.34,10737796145,114
2,AAP,160.22,10878565741,12
3,AAPL,127.80,2120186516103,15
4,ABBV,107.76,189996347802,18
5,ABC,99.97,20500359384,19
6,ABMD,265.82,11738671026,7
7,ABT,110.40,193783354147,17
8,ACN,246.62,163315982952,8
9,ADBE,484.06,231583175722,4


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [62]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe2.to_excel(writer, sheet_name='Recommended Trades', index= False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [63]:
background_color = '#0a0a23'
font_color = 'ffffff'
string_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

integer_format = writer.book.add_format({
    'num_format':'0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [64]:
writer.sheets['Recommeded Trades'].set_column('A:A',18, string_format)
writer.save()

KeyError: 'Recommeded Trades'

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [65]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Share Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of shares', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
writer.save()

## Saving Our Excel Output

Saving our Excel file is very easy:

In [1]:
git add .

SyntaxError: invalid syntax (<ipython-input-1-3d6cfb4a5e4d>, line 1)